In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [3]:
import os

os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java')
os.getcwd()

'/usr/local/lib/python3.10/dist-packages/konlpy/java'

In [4]:
# 임시 폴더에 Okt 사전 파일 압축 풀기
!pwd

/usr/local/lib/python3.10/dist-packages/konlpy/java


In [5]:
import shutil

filename = 'open-korean-text-2.1.0.jar'
src = '/content/drive/MyDrive/data/'
dir = '/usr/local/lib/python3.10/dist-packages/konlpy/java/'
shutil.copy(src + filename, dir + filename)

'/usr/local/lib/python3.10/dist-packages/konlpy/java/open-korean-text-2.1.0.jar'

In [6]:
# 테스트
from konlpy.tag import Okt
okt = Okt()
print(okt.nouns('문딸랑 좌좀비 문베 기집년닭근혜 레즈게이 더듬어강간당박사모'))
print(okt.morphs('문딸랑 좌좀비 문베 기집년닭근혜 레즈게이 더듬어강간당박사모'))

['문딸랑', '좌좀비', '문베', '기집년', '닭근혜', '레즈', '게이', '더듬어강간당', '박사모']
['문딸랑', '좌좀비', '문베', '기집년', '닭근혜', '레즈', '게이', '더듬어강간당', '박사모']


In [7]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib as mpl

path = '/content/drive/MyDrive/data/programmers_ai/yujatea/comp_2/'

In [8]:
train = pd.read_csv(os.path.join(path,'train.csv'))

In [9]:
vocab = pd.read_csv(os.path.join(path, 'vocab.csv'))

In [10]:
vocab

,key,replace
0,SiBal,시발
1,ssibark,시발
2,개sae들,개새들
3,Hannam,한남
4,sochu,소추
...,...,...
439,쪽 빠리,쪽바리
440,쉐 퀴,새끼
441,양아취새끼,양아치새끼
442,병 ㅅ들,병신들


In [11]:
vocab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   key      444 non-null    object
 1   replace  444 non-null    object
dtypes: object(2)
memory usage: 7.1+ KB


In [12]:
def text_preprocess(df, vocab):
  # 사용자 정의 지정 표현 수동으로 변환시켜주기
  for key, replace in zip(vocab['key'].values, vocab['replace'].values):
    try:
      df['text'] = df['text'].str.replace(key, replace, regex=True)
    except:
      print(key)

  # ') 깨문', '병@(#%^신' 처리
  df['text'] = df['text'].replace({r'\) 깨문': '대깨문'}, regex=True)
  df['text'] = df['text'].str.replace(r'병[@#%^]?\(*\)?.*?신', "병신", regex=True)

  # "OO"를 제외한 영어 표현은 EDA 결과 대체로 label 6에 해당 따라서 일괄적으로 토큰화 대상에서 제외 시켜준다.
  # 사실은 어떻게 핸들링을 해야할지 구체적인 방법을 모르겠다.
  # ?와 !는 어떤 의미를 가질것이라 추측하고 남긴다.
  # 나머지 특수기호들은 전부 지운다.
  df['text'] = df['text'].replace('[^0-9가-힣!?\sOO]+', '', regex=True)

  # '?'와 '!'가 연속해서 나오는 경우 1개만 남기고 지워주기
  df['text'] = df['text'].str.replace(r'!+', '!', regex=True)
  df['text'] = df['text'].str.replace(r'\?+', '?', regex=True)

  # 'ㅋㅋㅋ'나 'ㅎㅎㅎ' 처럼 한글 자음으로만 이루어진 경우는 지워주기
  df['text'] = df['text'].str.replace('[ㄱ-ㅎ]+', '?', regex=True)

  # 모음으로만 이루어진 부분들 지워주기
  df['text'] = df['text'].str.replace('[ㅏ-ㅣ]+', '?', regex=True)

In [13]:
def text_preprocess_test(df, vocab):
  # 사용자 정의 지정 표현 수동으로 변환시켜주기
  for key, replace in zip(vocab['key'].values, vocab['replace'].values):
    try:
      df['text'] = df['text'].str.replace(key, replace, regex=True)
    except:
      print(key)

  # ') 깨문', '병@(#%^신' 처리
  df['text'] = df['text'].replace({r'\) 깨문': '대깨문'}, regex=True)
  df['text'] = df['text'].str.replace(r'병[@#%^]?\(*\)?.*?신', "병신", regex=True)

  # '?'와 '!'가 연속해서 나오는 경우 1개만 남기고 지워주기
  df['text'] = df['text'].str.replace(r'!+', '!', regex=True)
  df['text'] = df['text'].str.replace(r'\?+', '?', regex=True)

In [14]:
# # 사용자 정의 지정 표현 수동으로 변환시켜주기
# for key, replace in zip(vocab['key'].values, vocab['replace'].values):
#   try:
#     train['text'] = train['text'].str.replace(key, replace, regex=True)
#   except:
#     print(key)

In [15]:
# ') 깨문', '병@(#%^신' 처리, 'X', 'x'
# train['text'] = train['text'].replace({r'\) 깨문': '대깨문'}, regex=True)
# train['text'] = train['text'].str.replace(r'병[@#%^]?\(*\)?.*?신', "병신", regex=True)

In [16]:
# 영어 표현은 EDA 결과 대체로 label 6에 해당 따라서 일괄적으로 토큰화 대상에서 제외 시켜준다
# 사실은 어떻게 핸들링을 해야할지 구체적인 방법을 모르겠다.
# ?와 !는 어떤 의미를 가질것이라 추측하고 남긴다.
# 나머지 특수기호들은 전부 지운다.
# train['text'] = train['text'].replace('[^0-9가-힣!?\s]+', '', regex=True)

In [17]:
# '?'와 '!'가 연속해서 나오는 경우 1개만 남기고 지워주기
# train['text'] = train['text'].str.replace(r'!+', '!', regex=True)

In [18]:
# train['text'] = train['text'].str.replace(r'\?+', '?', regex=True)

In [19]:
# 'ㅋㅋㅋ'나 'ㅎㅎㅎ' 처럼 한글 자음으로만 이루어진 경우는 지워주기
# train['text'] = train['text'].str.replace('[ㄱ-ㅎ]+', '?', regex=True)

In [20]:
# 모음으로만 이루어진 부분들 지워주기
# train['text'] = train['text'].str.replace('[ㅏ-ㅣ]+', '?', regex=True)

In [21]:
text_preprocess(train, vocab)

) 깨문
병@(#%^신


In [22]:
test = pd.read_csv(os.path.join(path, 'test.csv'))
test.head()

,ID,text,label
0,0,"""솔직히 우리나라 청년들도 불쌍하고 아재들도 불쌍하고 노인들도 불쌍하다. 나라가 참""",NaN
1,1,그만 보고싶네요 .늙은애들은.,NaN
2,2,"""더러운 개신교벌레 새퀴""",NaN
3,3,근데전태수씨 사망이유가뭔가요그어떤기사에도 나오질않네요,NaN
4,4,"""태극기부대와 틀닭바퀴충들에게 순시리는 국모다. ㅉㅉ""",NaN


In [23]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13491 entries, 0 to 13490
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      13491 non-null  int64  
 1   text    13491 non-null  object 
 2   label   0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 316.3+ KB


In [24]:
# 특수문자로만 이루어진 케이스
pattern = r'^[\W_]+$'
matches = test[test['text'].str.contains(pattern, regex=True)]
matches

,ID,text,label
1415,1415,"""👏👏👏👏👏👏👏🇰🇷🇰🇷🇰🇷🇰🇷🇰🇷🇰🇷🇰🇷""",NaN
12955,12955,"""🤣🤣🤣🤣🤣🤣🤣😋😋😋😋😋""",NaN
13402,13402,"""👏🏻👏🏻👏🏻👏🏻👏🏻""",NaN


In [25]:
matches_eng = test[test['text'].str.contains('[a-zA-Z]', regex=True)]
matches_eng

,ID,text,label
48,48,"""이희호같은 OOO 페미년도 거의 백살까지 살았는데 엔두는 120까지 가야지 ㄹㅇ ...",NaN
141,141,나도 요즘 나이가 좀 찬 30대라 막 챙겨보는사람은 아니고 처음에는BJ들 안좋게 봤...,NaN
143,143,"""Chinese cao ni ma !""",NaN
155,155,"""low 병 x 대댓글 자삭했네 ㅉㅉ 대깨문들은 걍 죽어줬으면 좋겠다""",NaN
231,231,"""전형적인 쪽발이 새xxx끼 관상이다. 하는짓도 생긴것과 똑같네""",NaN
...,...,...,...
13389,13389,"""OOO들 우리할머니 발뒤꿈치 보다 더 못생긴년들""",NaN
13425,13425,법을강화안시키니 이모양이꼴이지 뭐하는거니 도대체OOO,NaN
13454,13454,"""그뇬 이라고 한 걸레도 있었는데 저OOO 해도 되지모""",NaN
13482,13482,"""50대 OOO 들 왜 사냐 도대체 니들은""",NaN


In [26]:
text_preprocess_test(test, vocab)

) 깨문
병@(#%^신


In [27]:
stopwords = set()
f = open(os.path.join(path, 'stopwords.txt'), 'r')
lines = f.readlines()
for line in lines:
    line = line.strip().split()  # 줄 끝의 줄 바꿈 문자를 제거한다.
    for item in line:
      stopwords.add(item)
f.close()

print(len(stopwords))

680


In [28]:
# train['text'] = train['text'].apply(lambda x: ' '.join([i for i in okt.morphs(x) if i not in stopwords]))

In [29]:
# test['text'] = test['text'].apply(lambda x: ' '.join([i for i in okt.morphs(x) if i not in stopwords]))

In [30]:
print(train['text'].str.len().max())
print(test['text'].str.len().max())

299
301


In [31]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(train, test_size=0.2, random_state=42, stratify=train['label'])

In [32]:
train_df.shape, valid_df.shape

((52690, 3), (13173, 3))

In [33]:
# 토크나이저 정의
from konlpy.tag import Okt
okt = Okt()

In [34]:
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F

import random

In [35]:
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [36]:
# !pip install torchtext==0.6.0

In [37]:
ID = torchtext.data.Field(sequential = False,
                use_vocab = False) # 실제 사용은 하지 않을 예정

TEXT = torchtext.data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=okt.morphs, # 토크나이저로는 Mecab 사용.
                  # include_lengths = True,
                  batch_first=True,
                  stop_words=stopwords
                  )

LABEL = torchtext.data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)

In [38]:
torchtext.__version__

'0.6.0'

In [39]:
train_df.reset_index(inplace=True)
train_df = train_df.sort_values(by='ID')
train_df.drop(columns='index', inplace=True)
train_df.to_csv(os.path.join(path, 'train_df.csv'), index=False)

In [40]:
valid_df.reset_index(inplace=True)
valid_df = valid_df.sort_values(by='ID')
valid_df.drop(columns='index', inplace=True)
valid_df.to_csv(os.path.join(path, 'valid_df.csv'), index=False)

In [41]:
import numpy as np
test['label'] = np.random.randint(7)
test.to_csv(os.path.join(path, 'test_df.csv'), index=False)

In [42]:
# exception = pd.concat([matches, matches_eng])

# remain = test[~test['ID'].isin(list(exception['ID'].values))]
# exception = pd.concat([exception, remain[remain['text'] == '']])
# remain = remain[~remain['ID'].isin(list(exception['ID'].values))]
# remain.reset_index(inplace=True)
# remain = remain.sort_values(by='ID')
# remain.drop(columns='index', inplace=True)
# remain.to_csv(os.path.join(path,'remain.csv'),index=False)

# exception['label'] = 6

In [43]:
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
        path=path, train='train_df.csv', validation='valid_df.csv', test='test_df.csv', format='csv',
        fields=[('ID',ID),('text', TEXT), ('label', LABEL)], skip_header=True)

In [44]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

52690
13173
13491


In [45]:
train_df.shape, valid_df.shape, test.shape

((52690, 3), (13173, 3), (13491, 3))

In [46]:
print(vars(train_data[0]))
print(vars(valid_data[0]))
print(vars(test_data[0]))

{'ID': '0', 'text': ['유소영', '비호감', '성형', '아줌마'], 'label': '1'}
{'ID': '3', 'text': ['성희롱', '당할', '얼굴', '아닌데', '?'], 'label': '5'}
{'ID': '0', 'text': ['"', '솔직히', '우리나라', '청년', '도', '불쌍하고', '아재', '도', '불쌍하고', '노인', '도', '불쌍하다', '.', '나라', '"'], 'label': '0'}


In [47]:
TEXT.build_vocab(train_data, min_freq=1)
LABEL.build_vocab(train_data)
vocab_size = len(TEXT.vocab)
n_classes = 7
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 62388
클래스의 개수 : 7


In [48]:
BATCH_SIZE = 64


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    shuffle=True, 
    repeat=False, 
    sort=False,
    device = device
)

In [49]:
# iterator 출력
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iterator)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(valid_iterator)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iterator)))

훈련 데이터의 미니 배치의 개수 : 824
검증 데이터의 미니 배치의 개수 : 206
테스트 데이터의 미니 배치의 개수 : 211


In [50]:
batch = next(iter(train_iterator)) # 첫번째 미니배치
# print(batch.text[0].shape)
# print(batch.text[1].shape)
print(batch.text.shape)

torch.Size([64, 89])


In [51]:
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [52]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(device), batch.label.to(device)
        y= torch.tensor(y,dtype = torch.long,device =device)
        optimizer.zero_grad()

        logit = model(x)
        loss = criterion(logit, y)
        loss.backward()
        optimizer.step()

In [53]:
# 모델 평가 함수
def evaluate(model, val_iter):
    """검증 데이터셋 평가"""
    model.eval()
    batch_cor, total_loss = 0, 0

    for batch in val_iter:
        x, y = batch.text.to(device), batch.label.to(device)
        y= torch.tensor(y,dtype = torch.long,device =device)
        logit = model(x)

        loss = F.cross_entropy(logit, y)

        total_loss += loss.item()
        batch_cor += (logit.max(1)[1] == y.data).sum() 

    size = len(val_iter.dataset)
    avg_loss = total_loss / size 
    avg_accuracy = 100.0 * batch_cor / size


    return avg_loss, avg_accuracy

In [54]:
EPOCHS = 5

In [55]:
model = GRU(1, 256, vocab_size, 100, 7, 0.2).to(device)
lr = 0.0007
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.0001)
criterion = nn.CrossEntropyLoss() 

In [56]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iterator)
    val_loss, val_accuracy = evaluate(model, valid_iterator)

    print("[Epoch: %d] val loss : %5.3f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        torch.save(model.state_dict(), os.path.join(path, 'txtclassification.pt'))
        best_val_loss = val_loss

<ipython-input-52-48aac478aca6>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y= torch.tensor(y,dtype = torch.long,device =device)
<ipython-input-53-24877a702684>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y= torch.tensor(y,dtype = torch.long,device =device)


[Epoch: 1] val loss : 0.017 | val accuracy : 64.37
[Epoch: 2] val loss : 0.014 | val accuracy : 69.74
[Epoch: 3] val loss : 0.014 | val accuracy : 70.98
[Epoch: 4] val loss : 0.014 | val accuracy : 71.32
[Epoch: 5] val loss : 0.015 | val accuracy : 71.12


In [57]:
def predict(model, test_iter):
    """학습된 모델로 정답 파일 예측"""
    model.eval()
    preds = []
    pred = []
    for batch in test_iter:
        x = batch.text.to(device)
        y= torch.zeros(1)

        logit = model(x)
        pred=torch.argmax(logit,axis= 1).tolist()
        preds.extend(pred)
    return preds

In [58]:
submission = pd.read_csv(os.path.join(path,'submission.csv'))
submission.head()

,ID,label
0,0,NaN
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN


In [59]:
model.load_state_dict(torch.load(os.path.join(path, 'txtclassification.pt')))

<All keys matched successfully>

In [60]:
pred = predict(model,test_iterator)
len(pred)

13491

In [62]:
test['label'] = pred
test.to_csv(os.path.join(path,'my_submission.csv'),index=False)

In [63]:
test.drop(columns='text', inplace=True)

In [64]:
test

,ID,label
0,0,4
1,1,6
2,2,3
3,3,6
4,4,6
...,...,...
13486,13486,6
13487,13487,6
13488,13488,6
13489,13489,1


In [65]:
test.to_csv(os.path.join(path,'my_submission.csv'),index=False)